In [17]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Traffic matrices and server popularity distributions

In [59]:
def generate_traffic_matrix(sending, receiving):
    # normalize vectors
    sending = sending / np.linalg.norm(sending)
    receiving = receiving / np.linalg.norm(receiving)

    # sending and receiving are vectors of length n, reshape them to be nx1 and 1xn
    sending = np.reshape(sending, (len(sending), 1))
    receiving = np.reshape(receiving, (1, len(receiving)))
    
    # multiply the two vectors to get an nxn matrix 
    matrix = sending @ receiving
    np.fill_diagonal(matrix, 0)
    
    return matrix / np.sum(matrix)

Different distributions for server popularity

In [22]:
def generate_unif(n):
    # generate a uniform vector of length n
    return np.ones(n) / n

In [23]:
def generate_random(n):
    # generate a random vector of length n
    popularities = np.random.rand(n)
    return popularities / np.linalg.norm(popularities)

In [24]:
def generate_gaussian(n, std=0.1):
    # generate a uniform vector of length n, with some Gaussian noise added. default std is 0.1
    popularities = np.random.rand(n) + np.random.normal(0, std, n)
    return popularities / np.linalg.norm(popularities)

In [25]:
# Generate popularities according to Pareto dist

def generate_zipf(n, repeat=1):
    """Returns nodes' popularities according to Zipf dist
    ARGS:
        n: number of nodes
        repeat: number of nodes with 1/i popularity
        
    OUTPUTS:
        popularities: list of popularity for each node, following an approximate Zipf distribution
    """
    
    popularities = np.array([1/np.ceil(i/repeat) for i in range(1,n+1)]) # repeated Zipf function
    return popularities / np.linalg.norm(popularities) # normalize


# Jellyfish

In [44]:
def create_jellyfish(num_switches, other_switches_per_switch, servers_per_switch):
    # indices 0 to num_switches - 1 are switches
    G = nx.random_regular_graph(d=other_switches_per_switch, n=num_switches)
    
    # indices num_switches to num_switches + num_switches * servers_per_switch - 1 are servers
    for i in range(num_switches):
        for j in range(servers_per_switch):
            server_index = num_switches + i * servers_per_switch + j 
            G.add_node(server_index)
            G.add_edge(i, server_index)
            # servers.append(server_index)
    
    # list of all servers
    servers = range(num_switches, num_switches + num_switches * servers_per_switch)
    
    return G, servers


In [79]:
G, servers = create_jellyfish(num_switches=50, other_switches_per_switch=4, servers_per_switch=4)
# G, servers = create_jellyfish(num_switches=10, other_switches_per_switch=3, servers_per_switch=2)
print(servers)
print(nx.shortest_path_length(G, source=30, target=55))
# pos = nx.shell_layout(G)
# nx.draw(G, pos)
# plt.show()

range(50, 250)
6


In [78]:
def switch_weight(source, target, edge_attr):
    if source in servers or target in servers:
        return 1
    else:
        return 5

def distance_weight(source, target, edge_attr):
    if source in servers or target in servers:
        return 1
    else:
        return target-source
nx.shortest_path_length(G, source=130, target=155, weight=switch_weight)


6

In [80]:
def calculate_weighted_path_length(G, servers, edge_weight=None):
    num_servers = len(servers)
    servers = np.random.permutation(servers) # randomly permute servers

    shortest_path_lengths = np.zeros((num_servers, num_servers))
    for i in range(num_servers):
        for j in range(num_servers):
            shortest_path_lengths[i, j] = nx.shortest_path_length(G, source=servers[i], target=servers[j], weight=edge_weight)

    unif_traffic_matrix = generate_traffic_matrix(generate_unif(num_servers), generate_unif(num_servers)) 
    print("Uniform traffic", np.sum(np.multiply(shortest_path_lengths, unif_traffic_matrix)))
    random_traffic_matrix = generate_traffic_matrix(generate_random(num_servers), generate_random(num_servers))
    print("Random traffic", np.sum(np.multiply(shortest_path_lengths, random_traffic_matrix)))
    gaussian_traffic_matrix = generate_traffic_matrix(generate_gaussian(num_servers, std=0.25), generate_gaussian(num_servers, std=0.25))
    print("Gaussian traffic", np.sum(np.multiply(shortest_path_lengths, gaussian_traffic_matrix)))
    zipf_traffic_matrix = generate_traffic_matrix(generate_zipf(num_servers, repeat=5), generate_zipf(num_servers, repeat=5))
    print("Zipf traffic", np.sum(np.multiply(shortest_path_lengths, zipf_traffic_matrix)))

    

In [81]:
calculate_weighted_path_length(G, servers, edge_weight=switch_weight)

Uniform traffic 16.552763819095482
Random traffic 16.561651596693206
Gaussian traffic 16.58101197368703
Zipf traffic 16.593367065093634
